In [ ]:
import numpy as np
import pandas as pd
import tensorflow.compat.v2.feature_column as fc
import tensorflow as tf
from IPython.display import clear_output
from sklearn.model_selection import train_test_split

df = pd.read_csv('oasis_cross-sectional.csv')
df=df.drop(["Hand",'SES'], axis=1)
df=df.drop(["Delay"], axis=1)

df.dropna(how="any",inplace=True) #drop null rows
df['MMSE'] = df["MMSE"].astype(int)

dftrain, dftest = train_test_split(df, test_size=.2, random_state=21)
num_labels = df['MMSE'].unique()
y_train = dftrain.pop("MMSE")
y_test = dftest.pop("MMSE")

feature_columns = []
CATEGORICAL_COLUMNS = ['M/F']
NUMERIC_COLUMNS=['Age','Educ','CDR','eTIV','nWBV','ASF']

for col in CATEGORICAL_COLUMNS:
  vocabulary = dftrain[col].unique()
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(col,vocabulary))
for col in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(col,dtype=tf.float32))

def make_input_fn(train,labels,shuffle=False,num_epochs = 6,batch_size=64):
  def input_fn():
    ds = tf.data.Dataset.from_tensor_slices((dict(train),labels))
    if shuffle:
      ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_fn

train_fn = make_input_fn(dftrain,y_train,shuffle=True)
test_fn = make_input_fn(dftest,y_test,shuffle=False,num_epochs=1)

lin_est = tf.estimator.LinearClassifier(feature_columns=feature_columns,n_classes=31)

lin_est.train(train_fn)
result = lin_est.evaluate(test_fn)

clear_output()
print(result['accuracy'])
result = list(lin_est.predict(test_fn))


print(result[0],list(y_test)[0])

##Getting low accuracy, likely due to lack of comprehensive data values and small dataset

0.31914893
{'logits': array([-301.89496, -301.89496, -301.89496, -301.89496, -301.89496,
       -301.89496, -301.89496, -301.89496, -301.89496, -301.89496,
       -301.89496, -301.89496, -301.89496, -301.89496,  486.76266,
        598.09393,  413.01172,  660.8555 ,  443.75336,  427.42703,
        416.2052 ,  638.24194,  618.92584,  577.2086 ,  706.3458 ,
        630.41614,  597.5854 ,  651.35706,  838.8364 ,  891.90076,
        895.57965], dtype=float32), 'probabilities': array([0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       2.2176872e-25, 2.4628980e-02, 9.7537094e-01], d